In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython


In [2]:
%reload_ext Cython

In [3]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [4]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

### Load Pmpm, Canary data

In [5]:
%%time
canary_p = '../../../data/CANARY/CANARY202104'
pmpm_p = '../../../temp/PMPM.p'
preg_p = '../../../data/pregnancies'

preg = pq.read_table(preg_p).to_pandas()
canary = pq.read_table(canary_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)
canary.drop_duplicates(subset = ['PERS_ID'], inplace = True)

CPU times: user 5.56 s, sys: 7.26 s, total: 12.8 s
Wall time: 12.7 s


In [6]:
pmpm.MEDICAID_LOB.unique()

array(['Medicare', 'Commercial', 'Other', 'ASO', 'Medicaid-HARP',
       'Medicaid-TANF', 'Medicaid-SSI', 'Medicaid-CHP', 'Medicaid-Other'],
      dtype=object)

### Include Line of Bussiness Feature 1 Not medicaid 0 medicaid

In [7]:
medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
pmpm['LOB'] = 1
pmpm.loc[medicaid_mask, 'LOB'] = 0

In [8]:
preg = preg.rename(mapper = {'person_id':'PERS_ID'}, axis = 1)
preg = pd.melt(preg, id_vars = ['PERS_ID'], value_vars=['stay_start_date', 'stay_end_date'], value_name='MYR')
preg.MYR = preg.MYR.apply(lambda x: x.year*100+x.month)
preg.drop(labels = 'variable', axis = 1, inplace = True)
preg['PREGNANCY'] = 1

In [9]:
%%time
pmpm = pmpm.merge(preg, how = 'left', on = ['PERS_ID', 'MYR'])
pmpm.PREGNANCY = pmpm.PREGNANCY.fillna(value = 0)
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)

CPU times: user 12.1 s, sys: 7.24 s, total: 19.3 s
Wall time: 19.3 s


Keep Canary Patients from their date of registration and onwards who did not have **CC_DIABETES** flag on, on their date of registration.

In [12]:
%%time
canary2 = process_canary(canary, pmpm.MYR.max())
treated, controls = get_tc(pmpm.copy(), canary2)

CPU times: user 6.73 s, sys: 3.47 s, total: 10.2 s
Wall time: 10.2 s


In [13]:
%%time
treated2 = in_canary(treated.copy(), canary2)
#treated2 = filter_first_month_diabetes(treated2.copy())
treated2.drop(columns = ['IN_CANARY'], inplace = True)

CPU times: user 5.07 s, sys: 110 ms, total: 5.18 s
Wall time: 5.18 s


In [14]:
treated2.PERS_ID.unique().shape


In [170]:
treated_test  = treated2.groupby('PERS_ID').apply(lambda x: pd.Series(x.iloc[0]))

In [171]:
treated_test = treated_test[treated_test.MYR == 202007].reset_index(drop = True)

In [172]:
controls_test = controls[controls.MYR >= 202007].reset_index(drop = True)

In [173]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY', 'LOB',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT']

In [174]:
%%time

#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
hier_cols = None #['CC_DX_CNT']
hier_range = [1]
exact_match_dict = None#{'ACUTE':None, 'ACUTE6':None, 'ACUTE2':None, 'LOB':None}
nnr_func_args = {'match_on':columns_to_match, 'hier_cols':hier_cols,
                        'hier_range':hier_range, 'neighbors_per_treated': 1,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': True, 'exact_match_dict': exact_match_dict, 'restrict_time':False}
matched0, matched_data0 = matched_population(treated_test.copy(), 
                                           controls_test.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)

"""
#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
propensity_func_args = {'match_on':columns_to_match, 'neighbors_per_treated': 5, 'onset': "CC_DIABETES", "onset_val" = 1}
matched0, matched_data0 = matched_population(treated2.copy(), 
                                           controls.copy(),
                                           propensity_func, 
                                           propensity_func_args)
"""


Mathcing Function: nearest_neighbor_func
hERE
After filtering control ids are:204883 before they were:232569
CPU times: user 59.9 s, sys: 13.4 s, total: 1min 13s
Wall time: 1min 13s


'\n#columns_to_match = columns_to_match + [\'IP_BIN2\', \'ER_BIN2\']\npropensity_func_args = {\'match_on\':columns_to_match, \'neighbors_per_treated\': 5, \'onset\': "CC_DIABETES", "onset_val" = 1}\nmatched0, matched_data0 = matched_population(treated2.copy(), \n                                           controls.copy(),\n                                           propensity_func, \n                                           propensity_func_args)\n'

### keys have (id, month) tuples for the Treated for each (id, month) correspond a dataframe  with the neighbors

In [175]:
keys = list(matched0.keys())

### Join the controls into a  Matrix

In [176]:
dat1 = pd.DataFrame()
for key in keys:
    dat1 = pd.concat([dat1, matched0[key]], ignore_index = True)
    

In [177]:
dat2 = dat1[['PERS_ID', 'MYR']]
dat = pd.merge(dat2, controls_test, on = ['PERS_ID', 'MYR']).reset_index(drop = True)

### Here it is a design choice if you want to drop the duplicate controls (same id and month)

In [178]:
#dat.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)

####  In addition to the previously created matched groups create a random group

In [179]:
Path('./tempHybridSelectionMoreData').mkdir(exist_ok = True)
p = './tempHybridSelectionMoreData/'
matched_data0.to_pickle(p+'match0.p')
treated2.to_pickle(p+'treated0.p')

In [180]:
from utils import statistics_first_month

In [181]:
pmpm2 = pmpm
try:
    pmpm2.MBR_GNDR.replace(to_replace = {'F':0, 'M':1}, inplace = True)
except:
    pass

In [182]:
#treated_test['ALLOW_AMT'] = np.log(treated_test['ALLOW_AMT'].values+ 10**(-7)) 

In [183]:
%%time
tr_desc0, Ntr, treated_first_month = statistics_first_month(treated_test, columns_to_match, return_array = True)
tr_desc0 = tr_desc0.T



con_desc0, Ncon, controls_first_month = statistics_first_month(dat, columns_to_match, return_array = True)
#con_desc.index = pd.MultiIndex.from_product([['Matched'], con_desc.index])
con_desc0 = con_desc0.T



CPU times: user 112 ms, sys: 4.04 ms, total: 116 ms
Wall time: 116 ms


In [185]:
import scipy.stats as sts

In [186]:
args = {'mean1':tr_desc0['mean'], 'std1':tr_desc0['std'], 'mean2':con_desc0['mean'], 'std2':con_desc0['std'], 'nobs1': np.full(len(tr_desc0), Ntr) , 'nobs2': np.full(len(tr_desc0), Ncon)}
stats, pval = sts.ttest_ind_from_stats(**args)
stats = pd.DataFrame(stats, columns = ['t-stat'])
stats['pval'] = pval

In [187]:
alli = pd.concat((tr_desc0['mean'], con_desc0['mean'], stats), axis = 1, keys = ['Treated', 'Matched Controls', 'T-Test'] )

## Statistics on matching date

In [188]:
alli

Treated Matched Controls    T-Test          
                       mean             mean    t-stat      pval
AGE_AT_MIDMONTH   48.661290        48.677419 -0.007809  0.993782
ALLOW_AMT        669.145161       670.096774 -0.001971  0.998431
MBR_GNDR           0.161290         0.161290  0.000000  1.000000
CC_TOBACCO         0.048387         0.048387  0.000000  1.000000
CC_PRESSURE        0.000000         0.000000       NaN       NaN
CC_OBESITY         0.451613         0.451613  0.000000  1.000000
LOB                0.983871         0.983871  0.000000  1.000000
CC_HYPERTENSION    0.290323         0.290323  0.000000  1.000000
CC_HYPOTHYROID     0.064516         0.048387  0.386192  0.700027
CC_DX_CNT          2.322581         2.322581  0.000000  1.000000